# All imports

In [1]:
import pandas as pd

In [15]:
%pylab inline
from IPython.display import HTML
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from pypdb.clients.search.search_client import perform_search
from pypdb.clients.search.search_client import SearchService, ReturnType
from pypdb.clients.search.operators import text_operators

# Simple search by 'ATP-synthase'

Просто захотела проверить, как библиотека работает, что находит по словам

In [5]:
search_service = SearchService.TEXT
search_operator = text_operators.ContainsPhraseOperator(value="ATP-synthase",
                                            attribute="struct.title")
return_type = ReturnType.ENTRY

results = perform_search(search_service, search_operator, return_type)

print(results)


Querying RCSB Search using the following parameters:
 {"query": {"type": "terminal", "service": "text", "parameters": {"attribute": "struct.title", "operator": "contains_phrase", "value": "ATP-synthase"}}, "request_options": {"return_all_hits": true}, "return_type": "entry"} 

['5DN6', '6VMG', '6VMB', '6VMD', '6VM1', '6VM4', '6VOH', '6VOG', '6VOJ', '6VOI', '6VOL', '6VOK', '6VON', '6VOM', '6VOF', '6VOO', '6ZBB', '6ZNA', '6ZMR', '6TT7', '6PI4', '5FIK', '5FIJ', '5FIL', '6N2Y', '6N30', '5ARA', '5ARE', '5ARI', '5ARH', '6WNR', '6WNQ', '6ZA9', '6OQS', '6OQU', '6OQT', '6OQW', '6OQV', '6PQV', '6N2D', '6N2Z', '1L2P', '1C17', '5T4P', '5T4O', '5T4Q', '6ZQN', '6ZQM', '6ZG8', '6ZG7', '6ZIU', '6ZIQ', '6ZIT', '6ZIK', '6ZPO', '6OQR', '6CP6', '1FS0', '6B8H', '4V1G', '6YY0', '1AQT', '6Z1U', '6Z1R', '2CLY', '3U32', '2WPD', '5FL7', '6J5J', '6J5I', '6VWK', '3U2F', '3U2Y', '3V3C', '3UD0', '3OAA', '3ZO6', '2XND', '2WSS', '5BO5', '6TDV', '6RD4', '1B9U', '6YNX', '3P20', '3SSA', '3TGW', '3TIV', '4B2Q', '6J54', '

# Let's find all FoF1 complexes

## By structural similarity

Ищем все структуры, похожие на целую АТФ-синтазу

In [6]:
found_fof1_1 = Query('6FKF', query_type="structure").search()
set_fof1_level1 = set(found_fof1_1)
for i in found_fof1_1:
    found_i = Query(i, query_type="structure").search()
    set_fof1_level1.update(found_i)
    
print(set_fof1_level1)

{'6OQR', '6CP6', '6VOJ', '6CP3', '7JG6', '6VON', '5T4Q', '6A5O', '6OQV', '6VM1', '5ARI', '5T4P', '6OQT', '5LQY', '7JG5', '5FIL', '7JG8', '6WNR', '6VOL', '6OQU', '7JGA', '6FKH', '5ARA', '7JG9', '6ZPO', '6VMB', '5LQX', '5T4O', '6A5P', '6VOH', '6N2Y', '6FKF', '6J5I', '6OQW', '6VMG', '6PQV', '6I84', '6A5U', '6J5J', '6TT7', '5LQZ', '5DN6', '6WNQ', '6VOF', '6FKI', '7JG7', '6OQS', '5ARH', '6N30', '6N2Z', '5FIJ', '6ZQM', '6VM4', '5FIK', '6ZQN'}


In [7]:
set_fof1_level2 = set_fof1_level1.copy()
for i in set_fof1_level1:
    found_i = Query(i, query_type="structure").search()
    set_fof1_level2.update(found_i)
    
print(set_fof1_level2)

{'6A5R', '6CP3', '7JG6', '6J4Y', '5FIL', '7JG8', '7JGA', '6FKH', '6ZPO', '6A5P', '5LQX', '6RED', '6VOH', '6N2Y', '6FKF', '6J4Z', '6RER', '6REU', '6RDV', '6RDG', '6VOF', '7JG7', '6N30', '6ZQM', '6VM4', '5FIK', '6ZQN', '6INQ', '5T4Q', '6OQV', '5ARI', '5T4P', '5LQY', '6RDJ', '6J4W', '6A5L', '6VMG', '6J4X', '6I84', '6A5U', '6TT7', '5LQZ', '6IR9', '5DN6', '6FKI', '6N2Z', '6OQR', '6CP6', '6VOJ', '6A5O', '6J51', '5ARE', '6J50', '6RDY', '6OQU', '5ARA', '6RDB', '7JG9', '6VMB', '6A5T', '5T4O', '6OQW', '6J5J', '6WNQ', '5ARH', '6RDM', '6RDP', '6VON', '6VM1', '6OQT', '7JG5', '6WNR', '6VOL', '6J5I', '6PQV', '6RDS', '6OQS', '5FIJ'}


In [8]:
structure_table = {
    'pdb_id' : [],
    'title': [],
    'method': [],
    'year': [],
    'resolution' : [],
    'subunits': [],
    'ligands': [],
    'pubmed_id': []
}
for st in set_fof1_level2:
    info = get_info(st)
    if 'RNA' not in info['struct']['title']:
        structure_table['pdb_id'].append(st)
        structure_table['title'].append(info['struct']['title'])
        structure_table['method'].append(info['exptl'][0]['method'])
        structure_table['year'].append(info['citation'][0].get('year', []))
        structure_table['resolution'].append(info['rcsb_entry_info']['resolution_combined'][0])
        structure_table['subunits'].append(info['rcsb_entry_info']['polymer_entity_count_protein'])
        structure_table['ligands'].append(info['rcsb_entry_info'].get('nonpolymer_bound_components', []))
        structure_table['pubmed_id'].append(info['citation'][0].get('pdbx_database_id_pub_med', []))

In [9]:
df = pd.DataFrame(structure_table)
df.tail()

,pdb_id,title,method,year,resolution,subunits,ligands,pubmed_id
58,6J5I,Cryo-EM structure of the mammalian DP-state AT...,ELECTRON MICROSCOPY,2019,3.34,19,"[ADP, ATP, MG]",31197009
59,6PQV,E. coli ATP Synthase State 1e,ELECTRON MICROSCOPY,2020,3.30,8,"[ADP, ATP, MG, PO4]",32457314
60,6RDS,Cryo-EM structure of Polytomella F-ATP synthas...,ELECTRON MICROSCOPY,2019,3.80,7,"[ADP, ATP, MG]",31221832
61,6OQS,E. coli ATP synthase State 1b,ELECTRON MICROSCOPY,2020,3.30,8,"[ADP, ATP, MG, PO4]",32457314
62,5FIJ,Bovine mitochondrial ATP synthase state 2c,ELECTRON MICROSCOPY,2015,7.40,11,[],26439008


In [10]:
df.to_csv('fof1_structure_list_v2.csv')

# Let's find F1

In [11]:
found_f1_1 = Query('1E79', query_type="structure").search()
set_f1_level1 = set(found_f1_1)
for i in found_f1_1:
    found_i = Query(i, query_type="structure").search()
    set_f1_level1.update(found_i)
    
print(set_f1_level1)

{'3OEE', '2W6I', '4TSF', '1EFR', '2JDI', '2W6F', '6Q45', '3ZRY', '3VR4', '2F43', '2W6H', '4TT3', '1E1Q', '3A5C', '3OE7', '6VOO', '1COW', '6VOK', '6VOM', '6LY8', '6VOG', '3VR5', '5KNB', '2W6J', '2JJ1', '1JNV', '6SWE', '6VOI', '3W3A', '1W0K', '4Z1M', '3OFN', '2W6E', '3OAA', '6VMD', '2CK3', '1BMF', '1NBM', '3FKS', '4XD7', '4ASU', '2V7Q', '3OEH', '1OHH', '1H8E', '1E79', '1W0J', '3A5D', '2QE7', '5IK2', '2HLD', '5KND', '2JJ2', '6YO0', '3ZIA', '6FOC', '5HKK', '3VR6', '2JIZ', '1E1R', '2WSS', '5KNC', '4YXW', '1H8H', '6YY0', '2W6G', '1MAB'}


In [12]:
set_f1_level2 = set_f1_level1.copy()
for i in set_f1_level1:
    found_i = Query(i, query_type="structure").search()
    set_f1_level2.update(found_i)
    
print(set_f1_level2)

{'2W6I', '4TSF', '4GZY', '2W6H', '4TT3', '6BYU', '6RI9', '2W6J', '6JBQ', '4YLO', '3W3A', '5VSW', '4YFK', '3HKZ', '3FKS', '3VR2', '5UAJ', '6UT5', '1W0J', '6CHS', '6J9F', '6ALG', '5HKK', '1E1R', '2WSS', '5KNC', '3VR3', '3OEE', '6M6C', '1SMY', '6VOK', '6VOM', '6VOG', '5KNB', '6EYD', '2JJ1', '6SWE', '6ZOL', '2A69', '5UAQ', '1BMF', '2CK3', '6FBV', '5TBZ', '4YFN', '4XSZ', '2HLD', '6C6U', '2BE5', '6RIP', '3AOH', '3VR6', '6KF3', '5ZX3', '2CW0', '4KN7', '1EFR', '2JDI', '3OE7', '4ZH2', '1JNV', '6BZO', '6KOP', '1FX0', '5ZX2', '4BUJ', '3OAA', '6VMD', '4ASU', '4QIW', '2V7Q', '3OEH', '4MEX', '5W1T', '6VVT', '4YXW', '5W1S', '2A6E', '6LY8', '3OFN', '6P19', '4KN4', '4MQ9', '1E79', '4ZH4', '2QE7', '5IK2', '4ZH3', '6DCF', '5KND', '6FOC', '6RH3', '5VI5', '2Y0S', '2WAQ', '2W6F', '4KMU', '6Q45', '3ZRY', '6BJS', '1COW', '2A6H', '4YLN', '3VR5', '6NJO', '6CUX', '4Z1M', '1NBM', '4LJZ', '4XSY', '5UAH', '6N61', '1MAB', '6RI7', '1KMH', '2PMZ', '2F43', '1E1Q', '5ZE9', '6R9G', '2A68', '6VOI', '1W0K', '2W6E', '4XD7',

In [13]:
structures_f1_table = {
    'pdb_id' : [],
    'title': [],
    'method': [],
    'year': [],
    'resolution' : [],
    'subunits': [],
    'ligands': [],
    'pubmed_id': []
}
for st in set_f1_level2:
    info = get_info(st)
    if 'RNA' not in info['struct']['title']:
        structures_f1_table['pdb_id'].append(st)
        structures_f1_table['title'].append(info['struct']['title'])
        structures_f1_table['method'].append(info['exptl'][0]['method'])
        structures_f1_table['year'].append(info['citation'][0].get('year', []))
        structures_f1_table['resolution'].append(info['rcsb_entry_info']['resolution_combined'][0])
        structures_f1_table['subunits'].append(info['rcsb_entry_info']['polymer_entity_count_protein'])
        structures_f1_table['ligands'].append(info['rcsb_entry_info'].get('nonpolymer_bound_components', []))
        structures_f1_table['pubmed_id'].append(info['citation'][0].get('pdbx_database_id_pub_med', []))

df_f1 = pd.DataFrame(structures_f1_table)
df_f1.head()
df_f1.to_csv('f1_structure_list_v2.csv')


# Additional info

Можно посмотреть, как выглядит большая непонятная выдача в виде json'ины

In [88]:
import json
kk = json.dumps(info, indent=2)